### Higher Dimensional Example

While the previous example demonstrated the basic usage of the scan package,
one very important part still needs to be discussed: The hyperparameters of RC.

To do so, here we look at a much higher dimensional system than before, the
Lorenz-96 (L96) system, another chaotic system that, in constrast to the Lorenz-63
system, can be scaled to an arbitrarily large number of dimensions.
Here we scale it to be 40 dimensional.

As always we import the needed packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scan

and simulate the system. 

In [ ]:
simulation_time_steps = 100000
lor_dim = 40
lor_dt = 0.05
lor_force = 5

starting_point = lor_force * np.ones(lor_dim) + 1e-5 * np.random.rand(lor_dim)
sim_data = scan.simulations.Lorenz96(sys_dim=lor_dim, force=lor_force, dt=lor_dt).simulate(
    time_steps=simulation_time_steps, starting_point=starting_point
)

As before, we want to train the system with data sampled from the chaotic
attractor we want to predict. As a result,
we have to throw the first 10000 simulated data points away, as they are
not on said attractor, but instead correspond to transient system dynamics
of the L96 system.

In [ ]:
signal = sim_data[10000:]

An additional artifact of simulated systems like this, is that they are of
course perfectly noiseless. For higher dimensional systems, this can result
in RC "overfitting" the data, resulting in mostly accurate short term
predictions at the cost of many diverging completely after some time.

Somewhat counterintuitively, adding a small amount of artificial noise to
the training data can not only ensure the long term stability of the
prediction, but also decrease the short term prediction error.

Note that this procedure depends on the system in question, and is of
course not an issue when training with any even slightly noise real world
data.

In [ ]:
noise = np.random.normal(scale=0.01, size=signal.shape)
signal = signal + noise

Now we again want to create the reservoir/network, but this time the default
parameters are not sufficient for RC to learn the system dynamics. For the L96
system the following parameters work:

In [ ]:
n_dim = 5000  # network dimension
n_rad = 0.8  # spectral radius
n_avg_deg = 3  # average network degree

The more complicated and higher dimensional the system is, the larger the 
network n_dim needs to be.  
The spectral radius n_rad should be smaller than 1 and must be larger than 0.  
The average network degree n_avg_deg can, in theory, be everything in 
(0, n_dim]. Typically, sparse networks perform better than dense ones though.

So far no reliable heuristics exist for what the correct hyperparameter ranges
for an arbitrary system might be. Luckily, RC is usually so fast to train and
evaluate, that you can go over the entire reasonable range of hyperparameter values
with basically any hyperparameter optimization code of your choice, or even just a
simple grid search.

The random seed, set to generate the same network everytime, is an
important hyperparameter too as different random networks can vary hugely in
their prediction performance even if all other hyperparameters are the same.
Any trustworthy

Finally, create the network with those parameters.

In [ ]:
esn = scan.ESNWrapper()
esn.create_network(n_dim=n_dim, n_rad=n_rad, n_avg_deg=n_avg_deg, n_seed=0)

The train() and train_and_predict() methods too, have a set of hyperparameters one
needs to optimize, the most important of which for this system is the regularization
parameter which is typically somewhere between 1e-2 and 1e-10.

In [ ]:
reg_param = 1e-8

Addtionally, RC, like most machine learning methods, works best with normalized data. A mean of 0 and a standard
deviation of 1 are the default choices.

In [ ]:
signal_mean = np.mean(signal)
signal_std = np.std(signal)

signal_normalized = (signal - signal_mean) / signal_std

We also specify a nonlinear transformation to be used during the fitting of 
the output matrix, which vastly decreases number of failed, diverging 
predictions by breaking up harmful symmetries in the linear RC setup. For how 
and why see e.g.:  
[Herteux, J.; Räth, C. Breaking Symmetries of the Reservoir Equations in Echo State Networks. Chaos 2020, 30.](
https://aip.scitation.org/doi/10.1063/5.0028993)

In [ ]:
w_out_fit_flag = "linear_and_square_r"

Define the number of training/prediction time steps

In [ ]:
train_sync_steps = 1000
train_steps = 50000
pred_steps = 500

and train+predict the system using the above hyperparameters.

In [ ]:
y_pred, y_test = esn.train_and_predict(
    signal_normalized,
    train_sync_steps=train_sync_steps,
    train_steps=train_steps,
    pred_steps=pred_steps,
    reg_param=reg_param,
    w_out_fit_flag=w_out_fit_flag,
    w_in_seed=0,
)

Plot the results

In [ ]:
fig, axs = plt.subplots(3, 1, sharex="all", figsize=(9, 6), constrained_layout=True, dpi=300)

vmin = np.min(y_test)
vmax = np.max(y_test)

im = axs[0].imshow(y_test.T, aspect="auto", vmin=vmin, vmax=vmax)
axs[0].set_title("Simulation")

axs[1].imshow(y_pred.T, aspect="auto", vmin=vmin, vmax=vmax)
axs[1].set_title("Prediction")

axs[2].imshow(y_pred.T - y_test.T, aspect="auto", vmin=vmin, vmax=vmax)
axs[2].set_title("Difference between simulation and prediction")

axs[1].set_ylabel("dimension")
axs[2].set_xlabel("time steps")
fig.colorbar(im, ax=axs)
plt.show()